<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>TF IDF</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>BERT</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Константная-модель" data-toc-modified-id="Константная-модель-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Константная модель</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Light-GBM" data-toc-modified-id="Light-GBM-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Light GBM</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>XGBoost</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#Анализ-результатов" data-toc-modified-id="Анализ-результатов-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Анализ результатов</a></span></li></ul></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Определение токсичных комментариев

## Обзор данных

In [3]:
#при необходимости
!pip install langdetect

     |████████████████████████████████| 981 kB 1.3 MB/s eta 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=a65318edb01f0d3e3412d313c5c862aef816b80b440ef5e33d1a7e606b1a13f9
  Stored in directory: /home/jovyan/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [ ]:
#при необходимости
!pip install pytorch-pretrained-bert pytorch-nlp

In [7]:
#импортируем необходимые библиотеки
import pandas as pd
pd.set_option('display.float_format', '{:,.3f}'.format) #настройка, чтобы числа показывались с 3мя знаками после запятой и разделителем разрядов
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from sklearn.model_selection import train_test_split

from numpy.random import RandomState
from scipy import stats as st
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.simplefilter('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import f1_score 

from langdetect import detect

import spacy
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords ')

import transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig, BertModel
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification, BertForTokenClassification
from tqdm import tqdm, trange, notebook

import io

import xgboost as xgb
from catboost import CatBoostClassifier



In [23]:
#при необходимости
import sys
!{sys.executable} -m pip install spacy

!{sys.executable} -m spacy download en

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-07-24 00:11:58.825395: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-24 00:11:58.825678: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-24 00:12:06.714684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-07-24 00:12:06.714736: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-24 00:12:06.718478: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-9U6FB7H
2022-07-24 00:12:06.718632: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-9U6FB7H


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.0/en_core_web_sm-3.4.0-py3-none-any.whl (12.8 MB)
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-07-24 00:12:27.777114: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-07-24 00:12:27.777162: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-24 00:12:32.105892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-07-24 00:12:32.105927: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-24 00:12:32.109083: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: DESKTOP-9U6FB7H
2022-07-24 00:12:32.109173: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: DESKTOP-9U6FB7H


In [5]:
#читаем файлы, выводим информацию о датасете
#try - адрес у меня, except - адрес для проверки
try:
    data = pd.read_csv('datasets/toxic_comments.csv')
except FileNotFoundError:
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [6]:
# выводим общую информацию о данных
display(data.head(10))
print()
display(data.info())
print()
print('Доли пропусков')
display(data.isna().mean())
print()
print('Проверка наличия дублирующихся строк')
data.duplicated().sum()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None


Доли пропусков


text    0.000
toxic   0.000
dtype: float64


Проверка наличия дублирующихся строк


0

In [7]:
#оценим баланс классов
data['toxic'].value_counts(normalize=True)

0   0.898
1   0.102
Name: toxic, dtype: float64

Судя по выведенным данным язык сообщений английский, но проверим, так ли это для всех сообщений датасета.

In [8]:
%%time
list = []
list_lang = []
list_error = []
list_indexed = []
list_error_indexes = []
for i in range(len(data['text'])):
    try:
        if detect(data['text'][i]) != 'en':
            list.append(data['text'][i])
            list_lang.append(detect(data['text'][i]))
            list_indexed.append(i)
    except:
        list_error.append(data['text'][i])
        list_error_indexes.append(i)
        next
        
print(len(list))
print(len(list_error))

4287
21
Wall time: 11min 1s


In [9]:
len(list)/data.shape[0]

0.02686578388303639

In [10]:
pd.Series(list_lang).value_counts()

de    548
en    388
fr    322
af    310
so    242
et    234
id    232
cy    190
nl    183
tl    157
no    151
sv    147
da    142
it    129
tr    116
ca    107
es     93
pt     81
hu     79
ro     75
sw     63
vi     56
fi     56
hr     39
pl     37
sl     27
sk     23
cs     23
sq     18
lt     10
lv      3
el      1
ta      1
bg      1
mk      1
uk      1
ja      1
dtype: int64

In [11]:
list[0:20]

['REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski',
 'REDIRECT Talk:Frank Herbert Mason',
 "Oh, it's me vandalising?xD See here. Greetings,",
 'Azari or Azerbaijani? \n\nAzari-iranian,azerbaijani-turkic nation.',
 '86.29.244.57|86.29.244.57]] 04:21, 14 May 2007',
 'Future Perfect at Sunrise|☼]] 14:59, 16',
 "I'm afraid that's a broken link for me. -",
 'REDIRECT Talk:José Manuel Rojas',
 '2010]]\n[[User talk:Wikireader41/Archive4|Archive 5-Mar 15',
 'Valerie Poxleitner \n\nValeri Poxleitner, A.K.A. Lights. If',
 '|listas = Manos Family',
 'Barnes                  Aus     1                       1       8',
 '06:15, 19 Aug 2004 (UTC)',
 '"\nNo problem at all! \xa0(talk) "',
 "I've just seen that",
 'Batman  \n\nI am Batman. You are Spiderman. I win.',
 'WikiDon, STOP stalking me!',
 'Type 3 looks gorgeous )  (talk)',
 '|listas = Schaefer, Nolan',
 '"::I LOL\'d hardest at J.delanoy\'s. P Cobra \n\n"']

In [12]:
list_error

['"\n\n-———————————————————————————————————————————————————————————-\n\nSuck my dick. Thank you.  \xa0—talk/hist "',
 '1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n\n1999\n\n2000\n\n2001\n\n2002\n\n2003',
 'http://www.imdb.com/name/nm2551199/filmoseries#tt1327666 71.223.125.139',
 '193.61.111.53  15:00',
 '~ \n\n68.193.147.157',
 '[   ]\nhttp://www.smallholder.co.uk/news/878983.honey_storage/',
 '14:53,',
 '92.24.199.233|92.24.199.233]]',
 '"\n\n 199.209.144.211  "',
 'http://www.everytopicintheuniverseexceptchickens.com/ !!!!',
 'http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html',
 '"\n \'\'\'\'\'\' 2010/2013 "',
 'http://wikipediawehaveaproblem.com/evidence-of-cyberbullying-wp-editors-and-admins/  http://wikipediawehaveaproblem.com/evidence-of-cyberbullying-wp-editors-and-admins/',
 'http://goproxing.biz 82.103.128.115',
 'http://www.newswire.ca/fr/story/358197/domaine-pinnacle-ice-cider-now-available-in-new-european-and-asian-markets .',
 '] // 

**Выводы по данным:**  
1. В датасете 159571 строк и 2 столбца: тексты твитов и их классификация на токсичные (1) и нетоксичные (0).  
2. Пропуски в данных отсутствуют.  
3. Дублирующиеся строки отсутствуют.  
4. Есть явный дисбаланс классов: 10% токсичных текстов против 90% нетокчисных. Это необходимо будет учесть при разделении на выборки и обучении моделей.  
5. Основной язык тестов - английский. Но при этом есть также 4332 строки, определенные как другие языки, а также 21 текст, для которого язык определить не получилось (в основном это тексты со ссылками). Поскольку эти данные составляют всего около 3% от датасета, лучше всего удалить их.  
6. Тексты приведены в разных регистрах, содержат служебные символы (\n,\r), знаки препинания и т.п. Перед использованием в моделях тексты необходимо будет очистить.  



In [13]:
#удалим тексты, язык которых не определяется как английский
data = data.drop(index=list_indexed+list_error_indexes)

In [14]:
data.head(20)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


## Подготовка данных

Для подбора оптимального соотношения между скоростью подготовки данных, скоростью обучения и качеством моделей сделаем несколько вариантов подготовки данных:  

1. Очистка от служебных символов, знаков перпинаний и т.п. и лемматизация. 
*1.1. Векторизация с помощью метода TF-IDF.  
*1.2. Векторизация с помощью предобученной модели BERT. 
Поскольку лемматизация занимает достаточно много времени, это приедтся сделать на ограниченном датасете.  
2. Только очистка от служебных символов, знаков перпинаний и т.п. на полном датасете. Векторизация с помощью метода TF-IDF.  

In [8]:
#создадим функцию для очистки и лемматизации текстов
def text_prepare(text):
    #заменяем все символы кроме буквенных на пробелы, убираем лишниe пробелы
    text = re.sub(r'(?:\n|\r)', ' ', text)
    text = re.sub(r"[^'a-zA-Z]", ' ', text)
    text = " ".join(text.split())
    text = text.lower()
    
    #лемматизируем слова в тексте с помощью библиотеки spacy
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    text = nlp(text)
    text = " ".join([token.lemma_ for token in text])

    return text


In [9]:
#создадим функцию отдельно для очистки текстов, потому что лемматизация идёт долго

def text_cleaner(text):
    #заменяем все символы кроме буквенных на пробелы, убираем лишниe пробелы
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r'[^a-zA-Z]',' ',text)
    text = " ".join(text.split())
    text = text.lower()
    
    return text


In [17]:
 #проверяем работу функций
print(data.loc[0,'text'])

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


In [18]:
print(text_prepare(data.loc[0,'text']))
print()
print(text_cleaner(data.loc[0,'text']))

explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york dolls fac and please do not remove the template from the talk page since I be retire now

explanation why the edits made under my username hardcore metallica fan were reverted they weren t vandalisms just closure on some gas after i voted at new york dolls fac and please don t remove the template from the talk page since i m retired now


В целом функция справляется с задачей лемматизации и очистки данных.

Поскольку лемматизация требует большого количества времени, выделим для неё рандомный сэмпл из 5000 строк.

In [60]:
%%time
data_sample = data.sample(5000, random_state=42)
data_sample['lemms'] = [text_prepare(text) for text in data_sample['text']]
data_sample.head()

Wall time: 41min 3s


,text,toxic,lemms
120678,"""\n\nThis is not speculation, it is an absolut...",0,this be not speculation it be an absolute trut...
25601,"""\n\nThere has got to be a better term to use ...",0,there have get to be a well term to use than k...
147276,", 17 April 2015 (UTC)\n\nIncorrect again.. wow...",0,april utc incorrect again wow the equation and...
69001,This message is regarding the page Roller coas...,0,this message be regard the page roller coaster...
71868,Added the part of Harivamsa Purana \n\nI added...,0,add the part of harivamsa purana I add the ref...


In [71]:
#запись в файл, чтобы не перезапускать каждый раз
data_sample.to_csv('datasets/tweets_lemms_5000_new.csv') 

### TF IDF

In [23]:
#разобьем очищенные и лемматизированные данные на обучающую и тестовую выборки
features = data_sample['lemms']
target = data_sample['toxic']

features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.25,
                                                                            stratify=target,
                                                                            random_state=42)
print(features_train.shape, target_train.shape)
print(features_test.shape, target_test.shape)

(3750,) (3750,)
(1250,) (1250,)


In [24]:
#переведем слова в векторное представление с помощью метода TF-IDF
#включение в признаки биграмм не улучшает метрику
stop_words = set(stopwords.words('english'))
count_tfidf = TfidfVectorizer(stop_words=stop_words,ngram_range=(1, 1))
features_train = count_tfidf.fit_transform(features_train)
features_test = count_tfidf.transform(features_test)
print(features_train.shape)
print(features_test.shape)

(3750, 15243)
(1250, 15243)


Создадим датафрейм с очищенными, но не лемматизированными текстами.

In [25]:
%%time
#только очистка текстов
data_clean = data.copy()
data_clean['clean_texts'] = [text_cleaner(text) for text in data_clean['text']]
data_clean.head()

Wall time: 3.08 s


,text,toxic,clean_texts
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [67]:
#разобьем только очищенные, но не лемматизированные данные на обучающую и тестовую выборки
features_clean = data_clean['clean_texts']
target_clean = data_clean['toxic']

features_train_clean, features_test_clean, target_train_clean, target_test_clean = train_test_split(features_clean,
                                                                            target_clean,
                                                                            test_size=0.25,
                                                                            stratify=target_clean,
                                                                            random_state=42)

print(features_train_clean.shape, target_train_clean.shape)
print(features_test_clean.shape, target_test_clean.shape)

(116447,) (116447,)
(38816,) (38816,)


In [68]:
%%time
#переведем слова в векторное представление с помощью метода TF-IDF
stop_words = set(stopwords.words('english'))
count_tfidf = TfidfVectorizer(stop_words=stop_words,ngram_range=(1, 1))
features_train_clean = count_tfidf.fit_transform(features_train_clean)
features_test_clean = count_tfidf.transform(features_test_clean)
print(features_train_clean.shape)
print(features_test_clean.shape)

(116447, 140736)
(38816, 140736)
Wall time: 5.51 s


В данных отмечен существенный дисбаланс классов: доля целевого признака составляет всего 10%.  
Для устранения дисбаланса классов воспользуемся методом downsampling (поскольку данных достаточно много), а также гиперпараметром class_weight в самих моделях.  

Подготовим данные с downsampling на основе полного датасета с очищенными данными.

In [199]:
#разобьем только очищенные, но не лемматизированные данные на обучающую и тестовую выборки
features_downsample = data_clean['clean_texts']
target_downsample = data_clean['toxic']
features_train_downsample, features_test_downsample, target_train_downsample, target_test_downsample = train_test_split(features_downsample,
                                                                            target_downsample,
                                                                            test_size=0.25,
                                                                            stratify=target_downsample,
                                                                            random_state=42)

print(features_train_clean.shape, target_train_clean.shape)
print(features_test_clean.shape, target_test_clean.shape)

(116447, 140736) (116447,)
(38816, 140736) (38816,)


In [200]:
target_train_downsample.value_counts()

0    105131
1     11316
Name: toxic, dtype: int64

In [214]:
#подготовка данных для downsampling

len_toxic = (target_train_downsample == 1).sum()
features_train_downsample = pd.concat([features_train_downsample[target_train_downsample == 1]] + [features_train_downsample[target_train_downsample == 0]
                               .sample(len_toxic,random_state=42)])

target_train_downsample = pd.concat([target_train_downsample[target_train_downsample == 1]] + [target_train_downsample[target_train_downsample == 0]
                                                                                               .sample(len_toxic,random_state=42)])
                    
#перемешиваем данные
features_train_downsample, target_train_downsample = shuffle(features_train_downsample, target_train_downsample, random_state=42)
print(features_train_downsample.shape,target_train_downsample.shape )

(22632,) (22632,)


In [215]:
target_train_downsample.value_counts()

1    11316
0    11316
Name: toxic, dtype: int64

In [216]:
%%time
#переведем слова в векторное представление с помощью метода TF-IDF
stop_words = set(stopwords.words('english'))
count_tfidf = TfidfVectorizer(stop_words=stop_words,ngram_range=(1, 1))
features_train_downsample = count_tfidf.fit_transform(features_train_downsample)
features_test_downsample = count_tfidf.transform(features_test_downsample)
print(features_train_downsample.shape)
print(features_test_downsample.shape)

(22632, 49578)
(38816, 49578)
Wall time: 1.89 s


### BERT

Подготовим данные при помощи предобученной модели BERT на основании очищенного и лемматизированного сэмпла из 5000 строк

In [61]:
%%time
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenized = data_sample.iloc[0:4992]['lemms'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,max_length=512)))


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the 

Wall time: 9.01 s


In [62]:
max_len = len(max(tokenized, key=len))
print(max_len)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print(padded.shape)

512
(4992, 512)


In [63]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape


(4992, 512)

In [64]:
%%time
batch_size = 64
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/78 [00:00<?, ?it/s]

Wall time: 43min 18s


In [66]:
features_bert = np.concatenate(embeddings)
target_bert = data_sample.iloc[0:4992]['toxic']
features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(features_bert,
                                                                                                target_bert, 
                                                                                                test_size=0.25,
                                                                                                stratify=target_bert,
                                                                                                random_state=42)


In [78]:
print(features_train_bert.shape,target_train_bert.shape)
print(features_test_bert.shape,target_test_bert.shape)

(3744, 768) (3744,)
(1248, 768) (1248,)


Поскольку даже на сэмпле в 5000 строк подготовка данных заняла больше 1 часа, на полном датасете не запускаю.

## Обучение моделей

### Константная модель

In [71]:
#списки для записи работы моделей
result_lemms=[]
result_clean=[]
result_downsample=[]
result_bert=[]


In [72]:
#в качетве константной модели используем предсказание целевого класса 1
predictions_const = pd.Series(1,index=target_train.index)
f1_const = f1_score(target_train,predictions_const)
print('f1 на константной модели', f1_const)
result_lemms.append(f1_const)
result_clean.append(f1_const)
result_downsample.append(f1_const)
result_bert.append(f1_const)


f1 на константной модели 0.1853375272199371


### Логистическая регрессия

In [73]:
%%time
# модель для сокращенного лемматизированного набора данных
pipe_lr = Pipeline([('clf', LogisticRegression(class_weight = 'balanced',
                                               n_jobs=4, 
                                               random_state=42)
                ])
params_lr  = {'penalty':['l1','l2','elasticnet'],
                'C':range(1,50)
            }

grid_lr = GridSearchCV(estimator = pipe_lr,
                       param_grid=params_lr,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_lr.fit(features_train, target_train)
                    
print('Лучшие гиперпараметры:', grid_lr.best_params_)
print('Лучший f1 на лемматизированных данных в 5000 строк:',grid_lr.best_score_)
result_lemms.append(grid_lr.best_score_)


Лучшие гиперпараметры: {'C': 13, 'penalty': 'l2'}
Лучший f1 на лемматизированных данных в 5000 строк: 0.669568597642482
Wall time: 31.9 s


In [74]:
%%time
# модель для очищенного набора данных
# поскольку с подбором параметров модель работает очень долго, в качестве лучших гиперпараметрах примем те, 
#которые были подобраны на сэмпле
model = LogisticRegression(class_weight = 'balanced',n_jobs=4, random_state=42,C=13,penalty='l2')
model.fit(features_train_clean, target_train_clean)
f1 = np.mean(cross_val_score(model,features_train_clean, target_train_clean,cv=5,scoring='f1'))
print('f1 на очищенных данных:',f1)
result_clean.append(f1)


f1 на очищенных данных: 0.7621981190514512
Wall time: 21.2 s


In [219]:
%%time
# модель для downsampling
pipe_lr = Pipeline([('clf', LogisticRegression(class_weight = 'balanced',
                                               n_jobs=4, 
                                               random_state=42)
                ])
params_lr  = {'penalty':['l1','l2','elasticnet'],
                'C':range(1,50)
            }

grid_lr = GridSearchCV(estimator = pipe_lr,
                       param_grid=params_lr,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_lr.fit(features_train, target_train)
print('Лучшие гиперпараметры:', grid_lr.best_params_)
print('Лучший f1 на downsampling:',grid_lr.best_score_)
result_downsample.append(grid_lr.best_score_)
params_downsample.append(grid_lr.best_params_)

Лучшие гиперпараметры: {'C': 8, 'penalty': 'l2'}
Лучший f1 на downsampling: 0.8922016556079809
Wall time: 1min 56s


In [77]:
%%time
# модель для обрабоки BERT
pipe_lr = Pipeline([('clf', LogisticRegression(class_weight = 'balanced',
                                               n_jobs=4, 
                                               random_state=42)
                ])
params_lr  = {'penalty':['l1','l2','elasticnet'],
                'C':range(1,50)
            }

grid_lr = GridSearchCV(estimator = pipe_lr,
                       param_grid=params_lr,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_lr.fit(features_train, target_train)
print('Лучшие гиперпараметры:', grid_lr.best_params_)
print('Лучший f1 на лемматизированных данных в 5000 строк:',grid_lr.best_score_)
result_bert.append(grid_lr.best_score_)
params_bert.append(grid_lr.best_params_)


Лучшие гиперпараметры: {'C': 4, 'penalty': 'l2'}
Лучший f1 на лемматизированных данных в 5000 строк: 0.6929214849936483
Wall time: 13min 54s


### Случайный лес

In [79]:
%%time
# модель для сокращенного лемматизированного набора данных
pipe_rf = Pipeline([('clf',  RandomForestClassifier(class_weight = 'balanced', 
                                                    n_jobs=4,
                                                    random_state=42)
                ])
params_rf  = {'n_estimators':range(50, 100, 5),
                        'max_depth':range(1, 10)} 

grid_rf = GridSearchCV(estimator = pipe_rf,
                       param_grid=params_rf,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_rf.fit(features_train, target_train)


print('Лучшие гиперпараметры:',grid_rf.best_params_)
print('Лучший f1:', grid_rf.best_score_)
result_lemms.append(grid_rf.best_score_)


Лучшие гиперпараметры: {'max_depth': 9, 'n_estimators': 55}
Лучший f1: 0.49681170952908393
Wall time: 17.9 s


In [80]:
%%time
# модель для очищенного набора данных
pipe_rf = Pipeline([('clf',  RandomForestClassifier(class_weight = 'balanced', 
                                                    n_jobs=4,
                                                    random_state=42)
                ])
params_rf  = {'n_estimators':range(50, 100, 5),
                        'max_depth':range(1, 10)} 

grid_rf = GridSearchCV(estimator = pipe_rf,
                       param_grid=params_rf,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_rf.fit(features_train_clean, target_train_clean)


print('Лучшие гиперпараметры:',grid_rf.best_params_)
print('Лучший f1:', grid_rf.best_score_)
result_clean.append(grid_rf.best_score_)


Лучшие гиперпараметры: {'max_depth': 9, 'n_estimators': 90}
Лучший f1: 0.3588833641533447
Wall time: 9min 6s


In [220]:
%%time
# модель для downsampling

pipe_rf = Pipeline([('clf',  RandomForestClassifier(class_weight = 'balanced', 
                                                    n_jobs=4,
                                                    random_state=42)
                ])
params_rf  = {'n_estimators':range(50, 100, 5),
                        'max_depth':range(1, 10)} 

grid_rf = GridSearchCV(estimator = pipe_rf,
                       param_grid=params_rf,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_rf.fit(features_train_downsample,target_train_downsample)


print('Лучшие гиперпараметры:',grid_rf.best_params_)
print('Лучший f1 на downsampling:', grid_rf.best_score_)
result_downsample.append(grid_rf.best_score_)


Лучшие гиперпараметры: {'max_depth': 9, 'n_estimators': 90}
Лучший f1 на downsampling: 0.7997816449604916
Wall time: 1min 10s


In [82]:
%%time
# модель для обрабоки BERT
pipe_rf = Pipeline([('clf',  RandomForestClassifier(class_weight = 'balanced', 
                                                    n_jobs=4,
                                                    random_state=42)
                ])
params_rf  = {'n_estimators':range(50, 100, 5),
                        'max_depth':range(1, 10)} 

grid_rf = GridSearchCV(estimator = pipe_rf,
                       param_grid=params_rf,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_rf.fit(features_train_bert, target_train_bert)


print('Лучшие гиперпараметры:',grid_rf.best_params_)
print('Лучший f1:', grid_rf.best_score_)
result_bert.append(grid_rf.best_score_)


Лучшие гиперпараметры: {'max_depth': 7, 'n_estimators': 60}
Лучший f1: 0.671410251909552
Wall time: 1min 42s


### Light GBM

In [83]:
%%time
# модель для сокращенного лемматизированного набора данных
pipe_lgbm = Pipeline([('clf', lgb.LGBMClassifier(class_weight = 'balanced')
                ])
params_lgb  = {'objective':['binary'],
            'n_estimators':[10,100,500,1000,1500,2000],
            'max_depth':range(1,10,1),
            'learning_rate':[0.01,0.1,0.15,0.2,0.25,0.3]}

grid_lgb = GridSearchCV(estimator = pipe_lgbm,
                       param_grid=params_lgb,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_lgb.fit(features_train, target_train)


print('Лучшие гиперпараметры:', grid_lgb.best_params_)
print('Лучший f1:',grid_lgb.best_score_)
result_lemms.append(grid_lgb.best_score_)


Лучшие гиперпараметры: {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 500, 'objective': 'binary'}
Лучший f1: 0.5177029294102465
Wall time: 6min 33s


In [90]:
%%time
# модель для очищенного набора данных
# поскольку с подбором параметров модель работает очень долго, в качестве лучших гиперпараметрах примем те, 
#которые были подобраны на сэмпле
model_boost = lgb.LGBMClassifier(class_weight = 'balanced',
                                 n_estimators=1000,
                                 max_depth=7,
                                 learning_rate=0.15,
                                 objective='binary')

model_boost.fit(features_train_clean, target_train_clean)
f1_clean = np.mean(cross_val_score(model_boost,features_train_clean, target_train_clean,scoring='f1',cv=5))

print('f1:',f1_clean)
result_clean.append(f1_clean)


f1: 0.7614708234009909
Wall time: 5min 57s


In [221]:
%%time
# модель для downsampling
# поскольку с подбором параметров модель работает очень долго, в качестве лучших гиперпараметрах примем те, 
#которые были подобраны на сэмпле
model_boost = lgb.LGBMClassifier(class_weight = 'balanced',
                                 n_estimators=1000,
                                 max_depth=7,
                                 learning_rate=0.15,
                                 objective='binary')
model_boost.fit(features_train_downsample, target_train_downsample)
f1 = np.mean(cross_val_score(model_boost,features_train_downsample, target_train_downsample,scoring='f1',cv=5))
print('f1:',f1)
result_downsample.append(f1)


f1: 0.8881967831095225
Wall time: 56.8 s


In [89]:
%%time
# модель для обрабоки BERT

pipe_lgbm = Pipeline([('clf', lgb.LGBMClassifier(class_weight = 'balanced')
                ])
params_lgb  = {'objective':['binary'],
            'n_estimators':[10,100,500,1000],
            'max_depth':range(1,10,1),
            'learning_rate':[0.01,0.1,0.15,0.2,0.25]}

grid_lgb = GridSearchCV(estimator = pipe_lgbm,
                       param_grid=params_lgb,
                       scoring ='f1',
                       cv=5,
                       n_jobs=4
                      )


grid_lgb.fit(features_train_bert, target_train_bert)


print('Лучшие гиперпараметры:', grid_lgb.best_params_)
print('Лучший f1:',grid_lgb.best_score_)
result_bert.append(grid_lgb.best_score_)


Лучшие гиперпараметры: {'objective': 'binary', 'n_estimators': 1000, 'max_depth': 1, 'learning_rate': 0.2}
Лучший f1: 0.6724773368032964
Wall time: 1min 26s


### XGBoost

In [93]:
%%time
# модель для сокращенного лемматизированного набора данных
xgb_r = xgb.XGBClassifier(n_estimators = 100, seed = 123)
xgb_r.fit(features_train,target_train)
preds = xgb_r.predict(features_test)
cvs = cross_val_score(xgb_r,features_train,target_train,cv=5,scoring='f1')
print('f1:',np.mean(cvs))


result_lemms.append(np.mean(cvs)) 

f1: 0.5848833145520813
Wall time: 2.18 s


In [94]:
%%time
# модель для очищенного набора данных
xgb_r = xgb.XGBClassifier(n_estimators = 100, seed = 123)
xgb_r.fit(features_train_clean,target_train_clean)
cvs = cross_val_score(xgb_r,features_train_clean,target_train_clean,cv=5,scoring='f1')
print('f1:',np.mean(cvs))

result_clean.append(np.mean(cvs)) 

f1: 0.7120602840296606
Wall time: 1min 11s


In [222]:
%%time
# модель для downsampling
xgb_r = xgb.XGBClassifier(n_estimators = 100, seed = 123)
xgb_r.fit(features_train_downsample, target_train_downsample)
cvs = cross_val_score(xgb_r,features_train_downsample, target_train_downsample,cv=5,scoring='f1')
print('f1:',np.mean(cvs))

result_downsample.append(np.mean(cvs)) 

f1: 0.8639143140099715
Wall time: 17.2 s


In [96]:
%%time
# модель для обрабоки BERT
xgb_r = xgb.XGBClassifier(n_estimators = 100, seed = 123)
xgb_r.fit(features_train_bert,target_train_bert)
cvs = cross_val_score(xgb_r,features_train_bert,target_train_bert,cv=5,scoring='recall')
print('f1:',np.mean(cvs))

result_bert.append(np.mean(cvs)) 

f1: 0.5353041695146958
Wall time: 18.6 s


### CatBoost

In [98]:
%%time
# модель для сокращенного лемматизированного набора данных

cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=50, 
                                   max_depth=6, 
                                   learning_rate=0.5, 
                                   random_state=42)
cat_model.fit(features_train, target_train)
f1 = np.mean(cross_val_score(cat_model,features_train,target_train,cv=5,scoring='f1'))
print('F1:', f1)

result_lemms.append(f1) 

0:	learn: 0.2663657	total: 251ms	remaining: 12.3s
1:	learn: 0.4057971	total: 302ms	remaining: 7.25s
2:	learn: 0.5386064	total: 353ms	remaining: 5.52s
3:	learn: 0.5505618	total: 404ms	remaining: 4.64s
4:	learn: 0.5267176	total: 454ms	remaining: 4.08s
5:	learn: 0.5745856	total: 506ms	remaining: 3.71s
6:	learn: 0.5901639	total: 561ms	remaining: 3.45s
7:	learn: 0.5901639	total: 611ms	remaining: 3.21s
8:	learn: 0.5952813	total: 665ms	remaining: 3.03s
9:	learn: 0.6129032	total: 714ms	remaining: 2.85s
10:	learn: 0.6276596	total: 778ms	remaining: 2.76s
11:	learn: 0.6287744	total: 833ms	remaining: 2.64s
12:	learn: 0.6214286	total: 888ms	remaining: 2.53s
13:	learn: 0.6479860	total: 940ms	remaining: 2.42s
14:	learn: 0.6758148	total: 990ms	remaining: 2.31s
15:	learn: 0.6870748	total: 1.04s	remaining: 2.21s
16:	learn: 0.6981450	total: 1.09s	remaining: 2.12s
17:	learn: 0.7056856	total: 1.14s	remaining: 2.02s
18:	learn: 0.6969697	total: 1.19s	remaining: 1.94s
19:	learn: 0.7164179	total: 1.24s	remaini

In [99]:
%%time
# модель для очищенного набора данных
cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=50, 
                                   max_depth=6, 
                                   learning_rate=0.9, 
                                   random_state=42)
cat_model.fit(features_train_clean, target_train_clean)
f1 = np.mean(cross_val_score(cat_model,features_train_clean,target_train_clean,cv=5,scoring='f1'))
print('F1:', f1)

result_clean.append(f1) 

0:	learn: 0.3947258	total: 963ms	remaining: 47.2s
1:	learn: 0.5685566	total: 1.8s	remaining: 43.2s
2:	learn: 0.5970728	total: 2.62s	remaining: 41.1s
3:	learn: 0.6304954	total: 3.51s	remaining: 40.3s
4:	learn: 0.6509545	total: 4.37s	remaining: 39.4s
5:	learn: 0.6606964	total: 5.2s	remaining: 38.1s
6:	learn: 0.6723288	total: 6.05s	remaining: 37.2s
7:	learn: 0.6778293	total: 6.93s	remaining: 36.4s
8:	learn: 0.6826991	total: 7.8s	remaining: 35.5s
9:	learn: 0.6926803	total: 8.61s	remaining: 34.4s
10:	learn: 0.6970914	total: 9.42s	remaining: 33.4s
11:	learn: 0.6992723	total: 10.3s	remaining: 32.6s
12:	learn: 0.7040849	total: 11.1s	remaining: 31.6s
13:	learn: 0.7088959	total: 11.9s	remaining: 30.6s
14:	learn: 0.7113578	total: 12.7s	remaining: 29.7s
15:	learn: 0.7173648	total: 13.6s	remaining: 28.8s
16:	learn: 0.7219620	total: 14.4s	remaining: 27.9s
17:	learn: 0.7261668	total: 15.2s	remaining: 27.1s
18:	learn: 0.7313899	total: 16.1s	remaining: 26.2s
19:	learn: 0.7331738	total: 16.9s	remaining:

In [223]:
%%time
# модель для downsampling
cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=50, 
                                   max_depth=6, 
                                   learning_rate=0.9, 
                                   random_state=42)
cat_model.fit(features_train_downsample, target_train_downsample)
f1 = np.mean(cross_val_score(cat_model,features_train_downsample, target_train_downsample,cv=5,scoring='f1'))

print('F1:', f1)

result_downsample.append(f1) 

0:	learn: 0.5116279	total: 270ms	remaining: 13.2s
1:	learn: 0.6162044	total: 523ms	remaining: 12.5s
2:	learn: 0.6605844	total: 780ms	remaining: 12.2s
3:	learn: 0.7163148	total: 1.04s	remaining: 11.9s
4:	learn: 0.7288440	total: 1.29s	remaining: 11.7s
5:	learn: 0.7485818	total: 1.54s	remaining: 11.3s
6:	learn: 0.7772254	total: 1.8s	remaining: 11s
7:	learn: 0.7882048	total: 2.05s	remaining: 10.8s
8:	learn: 0.7955796	total: 2.31s	remaining: 10.5s
9:	learn: 0.7986345	total: 2.56s	remaining: 10.2s
10:	learn: 0.8081748	total: 2.82s	remaining: 10s
11:	learn: 0.8185213	total: 3.1s	remaining: 9.82s
12:	learn: 0.8256428	total: 3.35s	remaining: 9.53s
13:	learn: 0.8310004	total: 3.63s	remaining: 9.32s
14:	learn: 0.8359463	total: 3.9s	remaining: 9.09s
15:	learn: 0.8421410	total: 4.2s	remaining: 8.93s
16:	learn: 0.8484498	total: 4.47s	remaining: 8.67s
17:	learn: 0.8519498	total: 4.74s	remaining: 8.42s
18:	learn: 0.8546623	total: 5.02s	remaining: 8.2s
19:	learn: 0.8583004	total: 5.32s	remaining: 7.97s

In [103]:
%%time
# модель для обрабоки BERT
cat_model = CatBoostClassifier(eval_metric="F1", 
                                   iterations=50, 
                                   max_depth=6, 
                                   learning_rate=0.5, 
                                   random_state=42)
cat_model.fit(features_train_bert, target_train_bert)
f1 = np.mean(cross_val_score(cat_model,features_train_bert, target_train_bert,cv=5,scoring='f1'))
print('F1:', f1)

result_bert.append(f1) 

0:	learn: 0.3464567	total: 60.4ms	remaining: 2.96s
1:	learn: 0.4850746	total: 120ms	remaining: 2.88s
2:	learn: 0.5800712	total: 179ms	remaining: 2.81s
3:	learn: 0.6423841	total: 240ms	remaining: 2.77s
4:	learn: 0.6828479	total: 302ms	remaining: 2.71s
5:	learn: 0.7207488	total: 360ms	remaining: 2.64s
6:	learn: 0.7320872	total: 419ms	remaining: 2.58s
7:	learn: 0.7411945	total: 480ms	remaining: 2.52s
8:	learn: 0.7786260	total: 541ms	remaining: 2.46s
9:	learn: 0.7951807	total: 602ms	remaining: 2.41s
10:	learn: 0.8330871	total: 663ms	remaining: 2.35s
11:	learn: 0.8325926	total: 721ms	remaining: 2.28s
12:	learn: 0.8554745	total: 781ms	remaining: 2.22s
13:	learn: 0.8683068	total: 840ms	remaining: 2.16s
14:	learn: 0.8825215	total: 899ms	remaining: 2.1s
15:	learn: 0.8983051	total: 962ms	remaining: 2.04s
16:	learn: 0.9011299	total: 1.02s	remaining: 1.98s
17:	learn: 0.9113924	total: 1.08s	remaining: 1.92s
18:	learn: 0.9230769	total: 1.14s	remaining: 1.86s
19:	learn: 0.9275766	total: 1.2s	remainin

### Анализ результатов

In [224]:
results = pd.DataFrame({'models':['constant','logistic regression', 'random forest', 'light gbm', 'xgboost','catboost'],
                       'lemms_5000' : result_lemms,
                       'clean_data' : result_clean,
                       'downsampling' : result_downsample,
                       'BERT_4992':result_bert,
                        })
display(results)

,models,lemms_5000,clean_data,downsampling,BERT_4992
0,constant,0.185,0.185,0.185,0.185
1,logistic regression,0.670,0.762,0.892,0.693
2,random forest,0.497,0.359,0.800,0.671
3,light gbm,0.518,0.761,0.888,0.672
4,xgboost,0.585,0.712,0.864,0.535
5,catboost,0.605,0.734,0.858,0.617


Лучший результат на всех видах данных показала логистическая регрессия.  
Самый лучший результат: на данных с downsampling, далее - на общей выборке  с очищенными данными.  
Скорее всего, при увеличении размера сэмплов до 30-50 тыс. строк лемматизированные данные показали бы результат лучше, но это требует больших высичлительых мощностей.  
Протестируем результаты работы логистической регрессии на всех видах данных на лучших гиперпараметрах,  т.к.во всех случаях возможно переобучения модели.

## Тестирование лучшей модели

In [249]:
%%time
# модель для сокращенного лемматизированного набора данных
result_test = []
model = LogisticRegression(class_weight = 'balanced',n_jobs=4, random_state=42,C=13,penalty='l2')
model.fit(features_train, target_train)
f1 = f1_score(target_test, model.predict(features_test))
print('f1 на тестовых данных с лемматизацией:',f1)
result_test.append(f1)

f1 на тестовых данных с лемматизацией: 0.6244343891402715
Wall time: 1.06 s


In [250]:
%%time
# модель для очищенного набора данных

model = LogisticRegression(class_weight = 'balanced',n_jobs=4, random_state=42,C=13,penalty='l2')
model.fit(features_train_clean, target_train_clean)
f1 = f1_score(target_test_clean, model.predict(features_test_clean))
print('f1 на очищенных данных:',f1)
result_test.append(f1)

f1 на очищенных данных: 0.7562715199212986
Wall time: 4.24 s


In [251]:
%%time
# модель для downsampling

model = LogisticRegression(class_weight = 'balanced',n_jobs=4, random_state=42,C=12,penalty='l2')
model.fit(features_train_downsample, target_train_downsample)
f1 = f1_score(target_test_downsample, model.predict(features_test_downsample))
print('f1 на downsampling:',f1)
result_test.append(f1)

f1 на downsampling: 0.6588938714499253
Wall time: 1.82 s


In [254]:
%%time
# модель для обработке BERT

model = LogisticRegression(class_weight = 'balanced',n_jobs=4, random_state=42,C=4,penalty='l2')
model.fit(features_train_bert, target_train_bert)
f1 = f1_score(target_test_bert, model.predict(features_test_bert))
print('f1 на обработке BERT:',f1)
result_test.append(f1)

f1 на обработке BERT: 0.6789667896678967
Wall time: 4.32 s


In [255]:
results_test = pd.DataFrame({'parametrs':['lemms_5000','clean_data', 'downsampling', 'BERT_4992'],
                       'f1 test' : result_test})

display(results_test)

,parametrs,f1 test
0,lemms_5000,0.624
1,clean_data,0.756
2,downsampling,0.659
3,BERT_4992,0.679


Таким образом, модель показывает лучший результат на полном датасете с очищенными, но не лемматизированными данными данными с веторизацией по алгоритму TF IDF, и этот результат немного лучше требуемого (0,75).  
В качестве борьбы с дисбалансом классов лучше всего показывает себя применение гиперпараметра class_weight при обучении моделей. При обучении на данных с downsampling метрика на тестовой выборке получается ниже требуемой.

## Общий вывод

В ходе исследования модели были обучены на данных, подготовленных разными способами:
1. Очищенные и лемматизированные данные (на сэмпле из 5000 строк) с векторизацией с помощью алгоритма TF IDF.  
2. Только очищенные данные (на полном датасете) с векторизацией с помощью алгоритма TF IDF.  
3. Данные, сбалансированные методом downsampling (для корректировки дисбаланса классов), с векторизацией с помощью алгоритма TF IDF.
4. Только очищенные данные (на сэмпле из 5000 строк) с векторизацией с помощью предобученной модели BERT.
 
В связи с долгим временем обработки часть исследования (лемматизация, обработка данных моделью BERT) проведена только на сэмплах из 5000 строк.  

В процессе был обучен ряд моделей машинного обучения по следующим алгоритмам: логистическая регрессия, случайный лес, бустинг (light GBM, XGBoost, CatBoost).  
На всех видах данных лучший результат показала логистическая регрессия, лучшая метрика - на полных очищенных данных.  
Скорее всего при обработке данных моделью BERT на подных данных можно было бы получить метрику лучше, но это требует либо большего времени, либо больших вычислительных мощностей.  
На данном этапе необходимо определиться, что важнее: скорость обработки данных и обучения или точность модели.